In [1]:
import tensorflow.compat.v1 as tf
import numpy as np

In [12]:
# tf.compat.v1.disable_eager_execution()
# model = Seq_deepCpf1([3,5,7], [100, 70, 40], l_rate=0.001)


In [101]:
#TensorFlow config
conf = tf.ConfigProto()
tf.reset_default_graph()
weights_dict = {}

with tf.Session(config=conf) as sess:
#     sess.run()

    model = Seq_deepCpf1([3,5,7], [100, 70, 40], l_rate=0.001)
    init = tf.global_variables_initializer()
    saver = tf.train.Saver()
#     saver = tf.train.import_meta_graph('../dataset/aax9249_supplementary_code/DeepSpCas9_weight/PreTrain-Final-False-3-5-7-100-70-40-0.001-550-True-80-60.meta')
    sess.run(init)
    print(saver.restore(sess, ("../dataset/aax9249_supplementary_code/DeepSpCas9_weight/PreTrain-Final-False-3-5-7-100-70-40-0.001-550-True-80-60")))
#     print(sess.run("Output_Layer"))
#     sess.run(init)
#     with tf.variable_scope("hidden_layer2", reuse=True):
    
    for t in tf.trainable_variables():
#         print(sess.run(tf.trainable_variables()[0]).shape)
#         print(tf.trainable_variables()[0].name)
        weights_dict[t.name] = sess.run(t)

INFO:tensorflow:Restoring parameters from ../dataset/aax9249_supplementary_code/DeepSpCas9_weight/PreTrain-Final-False-3-5-7-100-70-40-0.001-550-True-80-60
None


In [108]:
weights_dict['Fully_Connected_Layer1/W_fcl1:0'].shape

(2790, 80)

In [69]:
import torch
import torch.nn as nn
import torch.nn.init as init
from torch.utils.data import DataLoader, TensorDataset, Dataset
from torch.autograd import Variable
from torch.nn import functional as F
from torch.utils.data.sampler import SubsetRandomSampler
import numpy as np
from collections import defaultdict

In [156]:
args = {
    'kernel_num': [100, 70, 40],
    'dropout': 0.0,
    'fc1_out': 80,
    'fc2_out': 60,
    'fc3_out': 1,
    'input_max_length':30,
    'k_max' : 1
}
model = SimpleCNN(args)

In [157]:
model.float()
model(train_set[3][0].view(1,30,4))

torch.Size([1, 100, 28, 1])
torch.Size([1, 100, 14, 1])
torch.Size([1, 70, 26, 1])
torch.Size([1, 70, 13, 1])
torch.Size([1, 40, 24, 1])
torch.Size([1, 40, 12, 1])


tensor([[0.0148]], grad_fn=<MmBackward>)

In [158]:
model.fc1.weight.shape

torch.Size([80, 2790])

In [104]:
model.conv1.weight = nn.Parameter(torch.FloatTensor( reshape_first_to_last(weights_dict['conv1_W:0'])), requires_grad=True)
model.conv1.bias = nn.Parameter(torch.FloatTensor(weights_dict['conv1_b:0']), requires_grad=True)
model.conv2.weight = nn.Parameter(torch.FloatTensor(reshape_first_to_last(weights_dict['conv2_W:0'])), requires_grad=True)
model.conv2.bias = nn.Parameter(torch.FloatTensor(weights_dict['conv2_b:0']), requires_grad=True)
model.conv3.weight = nn.Parameter(torch.FloatTensor(reshape_first_to_last(weights_dict['conv3_W:0'])), requires_grad=True)
model.conv3.bias = nn.Parameter(torch.FloatTensor(weights_dict['conv3_b:0']), requires_grad=True)

In [105]:
model.fc1.weight = nn.Parameter(torch.FloatTensor(weights_dict['Fully_Connected_Layer1/W_fcl1:0']), requires_grad=True)
model.fc1.bias = nn.Parameter(torch.FloatTensor(weights_dict['Fully_Connected_Layer1/B_fcl1:0']), requires_grad=True)
model.fc2.weight = nn.Parameter(torch.FloatTensor(weights_dict['Fully_Connected_Layer2/W_fcl2:0']), requires_grad=True)
model.fc2.bias = nn.Parameter(torch.FloatTensor(weights_dict['Fully_Connected_Layer2/B_fcl2:0']), requires_grad=True)
model.fc3.weight = nn.Parameter(torch.FloatTensor(weights_dict['Output_Layer/W_out:0']), requires_grad=True)


In [106]:
model.fc1.weight.shape

torch.Size([2790, 80])

In [97]:
def reshape_first_to_last(arr):
    shape = arr.shape
    rt = np.zeros((shape[3], shape[1], shape[2], shape[0]))
    for i0 in range(shape[3]):
        for i1 in range(shape[1]):
            for i2 in range(shape[2]):
                for i3 in range(shape[0]):
                    rt[i0][i1][i2][i3] = arr[i3][i1][i2][i0]
    return rt

In [99]:
model.conv1.weight.shape

torch.Size([100, 3, 4, 1])

In [155]:
class BasicModule(nn.Module):
    def __init__(self):
        super(BasicModule, self).__init__()
        self.model_name = str(type(self))

    def load(self, path):
        self.load_state_dict(torch.load(path))

    def save(self, path):
        torch.save(self.state_dict(), path)
        

class SimpleCNN(BasicModule):
    def __init__(self,args):
        super(SimpleCNN, self).__init__()
        # output size of fc layers
        out1 = args['fc1_out']
        out2 = args['fc2_out']
        out3 = args['fc3_out'] # output layer
        self.params = args
        self.k = args['k_max']
        
        # self.k = args['k'] # for k max pooling
        self.kernel_num = args['kernel_num']
        self.dropout = nn.Dropout(args['dropout'])
 
        
        # convolution layers
        filter_sizes = [3, 5, 7]
        self.conv1 = nn.Conv2d(1, self.kernel_num[0], (3, 4), bias=True, padding=0)
        self.conv2 = nn.Conv2d(1, self.kernel_num[1], (5, 4), bias=True, padding=0)
        self.conv3 = nn.Conv2d(1, self.kernel_num[2], (7, 4), bias=True, padding=0)
#         self.conv4 = nn.Conv2d(1, self.kernel_num[3], (9, 4), bias=True)
#         self.conv5 = nn.Conv2d(1, 100, (11, 4), bias=True)
        
        # fc layers
        self.fc1 = nn.Linear((sum([self.kernel_num[i] * (31-filter_sizes[i]) // 2 for i in range(len(filter_sizes))]))*self.k, out1, bias=True)
        self.fc2 = nn.Linear(out1, out2, bias=True)
        self.fc3 = nn.Linear(out2, out3, bias=False)
        

    def _conv_and_pool(self, x, conv):
        # x: (batch, 1, size, 4)
        x = conv(x)
        # x: (batch, kernel_num, H_out, 1)
        print(x.shape)
        x = F.relu(x)
        
        # x: (batch, kernel_num, H_out)
        x = F.max_pool2d(x, kernel_size=(2,1), stride=(2, 1))
#         x = self.kmax_pooling(x, 2, k=self.k)
        print(x.shape)
        x = x.view(x.size(0), x.size(1) * x.size(2))
        #  (batch, kernel_num * k)
        return x
    
    def setDropout(self, dropout):
        self.dropout = nn.Dropout(dropout)


    def forward(self, x):
        # x: (batch, size)
        # x: (batch, sentence_length, embed_dim)
        x = x.view(x.size(0),1, 30, 4)
        x1 = self._conv_and_pool(x, self.conv1)  # (batch, kernel_num * k)
       
        x2 = self._conv_and_pool(x, self.conv2)  # (batch, kernel_num * k)
        x3 = self._conv_and_pool(x, self.conv3)  # (batch, kernel_num * k)
#         x4 = self._conv_and_pool(x, self.conv4)  # (batch, kernel_num * k)
#         x5 = self._conv_and_pool(x, self.conv5)  # (batch, kernel_num * k)

        
        x = torch.cat((x1, x2, x3), 1)  # (batch, 4 * kernel_num * k)
        x = self.dropout(x)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.dropout(x)
#         x = 
#         logit = F.log_softmax(x, dim=1)
        return self.fc3(x)
    
    def kmax_pooling(self, x, dim, k):
        index = x.topk(k, dim = dim)[1].sort(dim = dim)[0]
        return x.gather(dim, index)

In [117]:
class DNADataset(Dataset):
    
    def __init__(self, data, size):
        super(DNADataset, self).__init__()
        self.size = size
        self.x = np.zeros((len(data),size, 4))
        self.y = []

        for I in range(len(data)):
            self.y.append(data[I][1])
            if type(data[0][0])==str:
                for i in range(size):
                    seq = data[I][0].lower()
                    # one hot encoding
                    pos = "acgt".find(seq[i])
                    if pos >= 0:
                        self.x[I][i][pos] = 1
            else:
                self.x[I] = data[I][0]
        self.x = torch.FloatTensor(self.x)
        self.y = torch.FloatTensor(self.y)
        
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, index):
        return self.x[index], self.y[index]
        
    # return a subset of dataset of given range
    def get_subset(self, start, end):
        
        return DNADataset([(self.x[i],self.y[i]) for i in range(start, end)], self.size)
    
#
def load_data(file):
    data = []
    with open(file,"r") as f:
        for line in f:
            line = line.split()
            data.append((line[0], float(line[1])))
    return data

In [121]:
#
train_set = DNADataset(load_data("../dataset/train_txt"),30)
test_set = DNADataset(load_data("../dataset/test_txt"),30)

In [3]:
# Model
length = 30

class Seq_deepCpf1(object):
    def __init__(self, filter_size, filter_num, node_1 = 80, node_2 = 40, l_rate = 0.005):
        self.inputs = tf.placeholder(tf.float32, [None, 1, length, 4])
        self.targets = tf.placeholder(tf.float32, [None, 1])
        self.is_training = tf.placeholder(tf.bool)
        def create_new_conv_layer(input_data, num_input_channels, num_filters, filter_shape, pool_shape, name):
            # setup the filter input shape for tf.nn.conv_2d
            conv_filt_shape = [filter_shape[0], filter_shape[1], num_input_channels,
                              num_filters]

            # initialise weights and bias for the filter
            weights = tf.Variable(tf.truncated_normal(conv_filt_shape, stddev=0.03),
                                              name=name+'_W')
            bias = tf.Variable(tf.truncated_normal([num_filters]), name=name+'_b')

            # setup the convolutional layer operation
            out_layer = tf.nn.conv2d(input_data, weights, [1, 1, 1, 1], padding='VALID')

            # add the bias
            out_layer += bias

            # apply a ReLU non-linear activation
            out_layer = tf.layers.dropout(tf.nn.relu(out_layer), 0.3, self.is_training)

            # now perform max pooling
            ksize = [1, pool_shape[0], pool_shape[1], 1]
            strides = [1, 1, 2, 1]
            out_layer = tf.nn.avg_pool(out_layer, ksize=ksize, strides=strides, 
                                       padding='SAME')
            return out_layer

        L_pool_0 = create_new_conv_layer(self.inputs, 4, filter_num[0], [1, filter_size[0]], [1, 2], name='conv1')
        L_pool_1 = create_new_conv_layer(self.inputs, 4, filter_num[1], [1, filter_size[1]], [1, 2], name='conv2')
        L_pool_2 = create_new_conv_layer(self.inputs, 4, filter_num[2], [1, filter_size[2]], [1, 2], name='conv3')
        with tf.variable_scope('Fully_Connected_Layer1'):
            layer_node_0 = int((length-filter_size[0])/2)+1
            node_num_0 = layer_node_0*filter_num[0]
            layer_node_1 = int((length-filter_size[1])/2)+1
            node_num_1 = layer_node_1*filter_num[1]
            layer_node_2 = int((length-filter_size[2])/2)+1
            node_num_2 = layer_node_2*filter_num[2]
            L_flatten_0 = tf.reshape(L_pool_0, [-1, node_num_0])
            L_flatten_1 = tf.reshape(L_pool_1, [-1, node_num_1])
            L_flatten_2 = tf.reshape(L_pool_2, [-1, node_num_2])
            L_flatten = tf.concat([L_flatten_0, L_flatten_1, L_flatten_2], 1, name='concat')
            node_num = node_num_0 + node_num_1 + node_num_2
            W_fcl1 = tf.get_variable("W_fcl1", shape=[node_num, node_1])
            B_fcl1 = tf.get_variable("B_fcl1", shape=[node_1])
            L_fcl1_pre = tf.nn.bias_add(tf.matmul(L_flatten, W_fcl1), B_fcl1)
            L_fcl1 = tf.nn.relu(L_fcl1_pre)
            L_fcl1_drop = tf.layers.dropout(L_fcl1, 0.3, self.is_training)

        with tf.variable_scope('Fully_Connected_Layer2'):
            W_fcl2 = tf.get_variable("W_fcl2", shape=[node_1, node_2])
            B_fcl2 = tf.get_variable("B_fcl2", shape=[node_2])
            L_fcl2_pre = tf.nn.bias_add(tf.matmul(L_fcl1_drop, W_fcl2), B_fcl2)
            L_fcl2 = tf.nn.relu(L_fcl2_pre)
            L_fcl2_drop = tf.layers.dropout(L_fcl2, 0.3, self.is_training)
            
        with tf.variable_scope('Output_Layer'):
            W_out = tf.get_variable("W_out", shape=[node_2, 1])#, initializer=tf.contrib.layers.xavier_initializer())
            B_out = tf.get_variable("B_out", shape=[1])#, initializer=tf.contrib.layers.xavier_initializer())
            self.outputs = tf.nn.bias_add(tf.matmul(L_fcl2_drop, W_out), B_out)

        # Define loss function and optimizer
        self.obj_loss =  tf.reduce_mean(tf.square(self.targets - self.outputs))
        self.optimizer = tf.train.AdamOptimizer(l_rate).minimize(self.obj_loss)  

# def Model_Finaltest(sess, TEST_X, filter_size, filter_num, if3d, model, args, load_episode, best_model_path):
#     test_batch = 500
#     test_spearman = 0.0
#     optimizer = model.optimizer
#     TEST_Z = np.zeros((TEST_X.shape[0], 1), dtype=float)
    
#     for i in range(int(np.ceil(float(TEST_X.shape[0])/float(test_batch)))):
#         Dict = {model.inputs: TEST_X[i*test_batch:(i+1)*test_batch], model.is_training: False}
#         TEST_Z[i*test_batch:(i+1)*test_batch] = sess.run([model.outputs], feed_dict=Dict)[0]
    
#     OUT = open("RANK_final_{}.txt".format(best_model_path.split('/')[1]), "a")
#     OUT.write("Testing final \n {} ".format(tuple(TEST_Z.reshape([np.shape(TEST_Z)[0]]))))
#     OUT.write("\n")
#     OUT.close()
    
#     return

# def preprocess_seq(data):
#     print("Start preprocessing the sequence done 2d")
#     length = len(data[0])
#     DATA_X = np.zeros((len(data),1,length,4), dtype=int)
#     print(np.shape(data), len(data), length)
#     for l in range(len(data)):
#         for i in range(length):
#             try: data[l][i]
#             except: print(data[l], i, length, len(data))
#             if data[l][i]in "Aa":
#                 DATA_X[l, 0, i, 0] = 1
#             elif data[l][i] in "Cc":
#                 DATA_X[l, 0, i, 1] = 1
#             elif data[l][i] in "Gg":
#                 DATA_X[l, 0, i, 2] = 1
#             elif data[l][i] in "Tt":
#                 DATA_X[l, 0, i, 3] = 1
#             else:
#                 print "Non-ATGC character " + data[i]
#                 sys.exit()
#     print("Preprocessing the sequence done")
#     return DATA_X
    
# def getseq(filenum):
#     param = parameters['%s'%filenum]
#     FILE = open(path+param[0], "r")
#     data = FILE.readlines()
#     data_n = len(data) - 1
#     seq = []
#     #CA = np.zeros((data_n, 1), dtype=int)
#     for l in range(1, data_n+1):
#         data_split = data[l].split()
#         seq.append(data_split[1])
#         #CA[l-1,0] = int(data_split[2])*100
#     FILE.close()

#     processed_full_seq = preprocess_seq(seq)
#     return processed_full_seq, seq
    
# #TensorFlow config
# conf = tf.ConfigProto()
# conf.gpu_options.allow_growth = True
# os.environ['CUDA_VISIBLE_DEVICES'] = '0'

# best_model_cv = 0.0
# best_model_list = []
# for best_model_path in best_model_path_list:
#     for modelname in os.listdir(best_model_path):
#         if "meta" in modelname:
#             best_model_list.append(modelname[:-5])
        
# TEST_X = []
# TEST_X_nohot = []
# for TEST_NUM in TEST_NUM_SET:
#     tmp_X, tmp_X_nohot = getseq(TEST_NUM)
#     TEST_X.append(tmp_X)
#     TEST_X_nohot.append(tmp_X_nohot)

# for index in range(len(best_model_list)):
#     best_model_path = best_model_path_list[index]
#     best_model = best_model_list[index]
#     valuelist = best_model.split('-')
#     fulllist = []
    
#     for value in valuelist:
#         if value == 'True':
#             value=True
#         elif value == 'False':
#             value=False
#         else:
#             try:
#                 value=int(value)
#             except:
#                 try:value=float(value)
#                 except: pass
#         fulllist.append(value)
    
#     print(fulllist[2:])
    
#     if fulllist[2:][-3] is True:
#         if3d, filter_size_1, filter_size_2, filter_size_3, filter_num_1, filter_num_2, filter_num_3, l_rate, load_episode, inception, node_1, node_2 = fulllist[2:]
#         filter_size = [filter_size_1, filter_size_2, filter_size_3]
#         filter_num  = [filter_num_1, filter_num_2, filter_num_3]
#     else:
#         if3d, filter_size, filter_num, l_rate, load_episode, inception, node_1, node_2 = fulllist[2:]
#     args = [filter_size, filter_num, l_rate, load_episode]
#     tf.reset_default_graph()
#     with tf.Session(config=conf) as sess:
#         sess.run(tf.global_variables_initializer())
#         model = Seq_deepCpf1(filter_size, filter_num, node_1, node_2, args[2])
        
#         saver = tf.train.Saver()
#         saver.restore(sess, best_model_path + best_model)
        
#         OUT = open("RANK_final_{}.txt".format(best_model_path.split('/')[1]), "a")
#         OUT.write("{}".format(best_model))
#         OUT.write("\n")
#         OUT.close()
        
#         TEST_Y = []
        
#         for i in range(len(TEST_NUM_SET)):
#             print ("TEST_NUM : {}".format(TEST_NUM_SET[i]))
            
#             OUT = open("RANK_final_{}.txt".format(best_model_path.split('/')[1]), "a")
#             OUT.write("\n")
#             OUT.write("TEST_FILE : {}".format(parameters['{}'.format(TEST_NUM_SET[i])]))
#             OUT.write("\n")
#             OUT.close()
#             Model_Finaltest(sess, TEST_X[i], filter_size, filter_num, if3d, model, args, load_episode, best_model_path)

#         OUT = open("RANK_final_{}.txt".format(best_model_path.split('/')[1]), "a")
#         OUT.write("\n")
#         OUT.close()